In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/df_agri_keywords.csv')

In [3]:
df.head()

,channel_title,channel_name,country,segment_id,segment_start,segment_end,duration_seconds,num_keywords,avg_density,themes,report_text,category,secondary_categories,text_type,keywords,keywords_filtered,nouns_in_keywords
0,France 2,france2,france,5788,2025-12-03 06:34:04.031 +0100,2025-12-03 06:36:02.025 +0100,118,18,13.4,", attenuation_climatique_solutions_indirectes,...",plus vendu au monde a la une également cette c...,agriculture_alimentation,['energy'],segment,"['voyez', 'radiologues', 'entraîne activement ...","['entraîne activement lucie robert', 'stations...","['concurrence', 'stations', 'résultats', 'dire..."
1,M6,m6,france,3445,2025-11-17 15:20:24.048 +0100,2025-11-17 15:21:26.064 +0100,62,4,5.3,", biodiversite_causes_indirectes, changement_c...",christo popov thomas fersen ces enseignants et...,agriculture_alimentation,[],segment,"['enseigne travaille', 'va voir si', 'david', ...","['enseigne travaille', 'cours', 'prochaine car...","['temps', 'directeurs', 'retraites', 'site', '..."
2,M6,m6,france,1379,2025-05-13 20:14:11.059 +0200,2025-05-13 20:15:50.004 +0200,99,8,8.1,"attenuation_climatique_solutions_indirectes, b...",la baignade tourne court sera <unk> occasion d...,agriculture_alimentation,[],segment,"['réglementations strictes imposées', 'essaie'...","['réglementations strictes imposées', 'parc na...","['heure', 'mérite', 'sympathie', 'euros', 'rég..."
3,TF1,tf1,france,4852,2025-12-25 09:11:59.056 +0100,2025-12-25 09:13:11.082 +0100,72,15,17.2,"adaptation_climatique_solutions, attenuation_c...",le jardin ce ne sont pas uniquement des matièr...,agriculture_alimentation,[],segment,"['allez pouvoir', 'froid', 'herbes', 'myrtille...","['certains papiers', 'plantes', 'déchets', 'hu...","['année', 'pommes', 'andalou', 'choses', 'limi..."
4,France 2,france2,france,5744,2025-11-30 07:36:06.090 +0100,2025-11-30 07:38:37.058 +0100,151,11,10.5,"biodiversite_causes, biodiversite_concepts_gen...",aux etats-unis que pensez-vous d' aller faire ...,agriculture_alimentation,[],segment,"['voyez', 'côté alpes', 'météo', 'peu gêné', '...","['côté alpes', 'météo', 'extérieur', 'choses',...","['météo', 'choses', 'extérieur', 'règles', 'en..."


In [10]:
df['nouns_in_keywords']

0       ['concurrence', 'stations', 'résultats', 'dire...
1       ['temps', 'directeurs', 'retraites', 'site', '...
2       ['heure', 'mérite', 'sympathie', 'euros', 'rég...
3       ['année', 'pommes', 'andalou', 'choses', 'limi...
4       ['météo', 'choses', 'extérieur', 'règles', 'en...
                              ...                        
2560    ['pontes', 'taureau', 'coupe', 'autrefois', 'm...
2561    ['alvin', 'miracle', 'temps', 'fonds', 'pannea...
2562    ['heure', 'instabilité', 'pays', 'monsieur', '...
2563    ['barrière', 'soulage', 'oesophage', 'mg', 'to...
2564    ['pense', 'temps', 'valentine', 'extérieur', '...
Name: nouns_in_keywords, Length: 2565, dtype: object

In [11]:
"""
Classification des discours agricoles en 4 catégories basées sur les mots-clés :

1. PRODUCTIVISTE : Agriculture intensive, industrielle, orientée rendement
   - Focus sur : rendement, production de masse, pesticides, engrais chimiques,
     mécanisation, exportation, agro-industrie, OGM, monoculture

2. ALTERNATIF : Agriculture durable, bio, agroécologie
   - Focus sur : bio, agroécologie, permaculture, biodiversité, circuits courts,
     agriculture paysanne, semences paysannes, agroforesterie, sans pesticides

3. NEUTRE : Discours agricole général sans orientation claire
   - Termes généraux : agriculteur, ferme, récolte, terrain, culture,
     exploitation, saison, météo, marché

4. AUTRE : Ne parle pas d'agriculture
   - Absence de vocabulaire agricole significatif
"""

import ast
from typing import List, Literal

# Dictionnaires de mots-clés pour chaque catégorie
KEYWORDS_PRODUCTIVISTE = {
    # Intensification & rendement
    'rendement', 'productivité', 'production', 'intensif', 'intensive', 'industriel', 
    'industrielle', 'agro-industrie', 'agroindustrie', 'usine', 'exportation', 'export',
    # Intrants chimiques
    'pesticide', 'pesticides', 'herbicide', 'herbicides', 'fongicide', 'fongicides',
    'insecticide', 'insecticides', 'engrais', 'chimique', 'chimiques', 'phytosanitaire',
    'phytosanitaires', 'traitement', 'traitements', 'glyphosate', 'roundup',
    # Pratiques intensives
    'monoculture', 'ogm', 'transgénique', 'mécanisation', 'irrigation', 'drainage',
    'remembrement', 'concentré', 'élevage intensif', 'batterie', 'feedlot',
    # Économie de marché
    'compétitivité', 'concurrence', 'mondialisation', 'coopérative', 'négoce',
    'subvention', 'pac', 'quota', 'cours', 'bourse', 'spéculation',
    # Acteurs industriels
    'monsanto', 'bayer', 'syngenta', 'lactalis', 'bigard', 'avril', 'sofiprotéol'
}

KEYWORDS_ALTERNATIF = {
    # Agriculture biologique
    'bio', 'biologique', 'biologiques', 'organique', 'organiques', 'label', 'certification',
    'ab', 'agriculture biologique', 'conversion',
    # Agroécologie & permaculture
    'agroécologie', 'agroécologique', 'permaculture', 'agroforesterie', 'polyculture',
    'rotation', 'association', 'compagnonnage', 'couvert', 'couverts', 'engrais vert',
    # Biodiversité & environnement
    'biodiversité', 'écosystème', 'pollinisateur', 'pollinisateurs', 'abeille', 'abeilles',
    'haie', 'haies', 'bocage', 'mare', 'zone humide', 'sol vivant', 'humus',
    # Circuits courts & local
    'circuit court', 'circuits courts', 'local', 'locaux', 'proximité', 'amap',
    'vente directe', 'marché paysan', 'ferme', 'terroir', 'artisanal',
    # Agriculture paysanne
    'paysan', 'paysanne', 'paysans', 'petit producteur', 'semences paysannes',
    'autonomie', 'autosuffisance', 'souveraineté alimentaire',
    # Sans intrants
    'sans pesticide', 'zéro phyto', 'naturel', 'naturels', 'compost', 'fumier',
    'auxiliaire', 'auxiliaires', 'lutte biologique', 'prédateur', 'coccinelle',
    # Bien-être animal
    'bien-être animal', 'plein air', 'pâturage', 'extensif', 'extensive', 'herbe'
}

KEYWORDS_AGRICULTURE_NEUTRE = {
    # Termes généraux agriculture
    'agriculture', 'agricole', 'agricoles', 'agriculteur', 'agriculteurs', 'agricultrice',
    'exploitation', 'exploitant', 'cultivateur', 'producteur', 'producteurs',
    # Cultures & élevage
    'culture', 'cultures', 'récolte', 'récoltes', 'moisson', 'semis', 'plantation',
    'céréale', 'céréales', 'blé', 'maïs', 'orge', 'colza', 'tournesol', 'betterave',
    'légume', 'légumes', 'fruit', 'fruits', 'vigne', 'vignoble', 'vin', 'olive',
    'élevage', 'éleveur', 'bétail', 'vache', 'vaches', 'bovin', 'bovins', 'lait',
    'porc', 'porcs', 'volaille', 'volailles', 'poulet', 'mouton', 'moutons', 'ovin',
    # Infrastructure
    'ferme', 'grange', 'silo', 'tracteur', 'machine', 'matériel', 'terrain', 'parcelle',
    'champ', 'champs', 'prairie', 'pré', 'verger', 'serre',
    # Saisons & météo
    'saison', 'printemps', 'été', 'automne', 'hiver', 'sécheresse', 'gel', 'grêle',
    'pluie', 'météo', 'climat', 'température',
    # Économie agricole
    'marché', 'prix', 'vente', 'achat', 'revenu', 'aide', 'filière'
}


def parse_keywords(keywords_str: str) -> List[str]:
    """Parse la chaîne de caractères des keywords en liste."""
    if pd.isna(keywords_str) or keywords_str == '':
        return []
    try:
        # Essayer de parser comme une liste Python
        return ast.literal_eval(keywords_str)
    except (ValueError, SyntaxError):
        # Si échec, retourner une liste vide
        return []


def classify_discourse(keywords: List[str]) -> dict:
    """
    Classifie un discours basé sur ses mots-clés.
    
    Args:
        keywords: Liste de mots-clés (noms) extraits du discours
    
    Returns:
        dict avec:
            - category: la catégorie principale
            - scores: les scores pour chaque catégorie
            - matched_keywords: les mots-clés matchés par catégorie
    """
    if not keywords:
        return {
            'category': 'autre',
            'scores': {'productiviste': 0, 'alternatif': 0, 'neutre': 0, 'autre': 1},
            'matched_keywords': {'productiviste': [], 'alternatif': [], 'neutre': []}
        }
    
    # Normaliser les keywords (lowercase)
    keywords_lower = [k.lower().strip() for k in keywords]
    
    # Compter les matches pour chaque catégorie
    matched = {
        'productiviste': [],
        'alternatif': [],
        'neutre': []
    }
    
    for kw in keywords_lower:
        # Vérifier productiviste
        if kw in KEYWORDS_PRODUCTIVISTE or any(p in kw for p in KEYWORDS_PRODUCTIVISTE):
            matched['productiviste'].append(kw)
        # Vérifier alternatif
        if kw in KEYWORDS_ALTERNATIF or any(a in kw for a in KEYWORDS_ALTERNATIF):
            matched['alternatif'].append(kw)
        # Vérifier neutre (agriculture générale)
        if kw in KEYWORDS_AGRICULTURE_NEUTRE or any(n in kw for n in KEYWORDS_AGRICULTURE_NEUTRE):
            matched['neutre'].append(kw)
    
    # Calculer les scores (pondérés)
    # Productiviste et Alternatif ont plus de poids car plus spécifiques
    score_prod = len(matched['productiviste']) * 2
    score_alt = len(matched['alternatif']) * 2
    score_neutre = len(matched['neutre'])
    
    total_agri = score_prod + score_alt + score_neutre
    
    # Si aucun terme agricole, c'est "autre"
    if total_agri == 0:
        return {
            'category': 'autre',
            'scores': {'productiviste': 0, 'alternatif': 0, 'neutre': 0, 'autre': 1},
            'matched_keywords': matched
        }
    
    # Déterminer la catégorie
    scores = {
        'productiviste': score_prod / total_agri if total_agri > 0 else 0,
        'alternatif': score_alt / total_agri if total_agri > 0 else 0,
        'neutre': score_neutre / total_agri if total_agri > 0 else 0,
        'autre': 0
    }
    
    # Si productiviste ou alternatif dominent clairement
    if score_prod > score_alt and score_prod > score_neutre * 0.5:
        category = 'productiviste'
    elif score_alt > score_prod and score_alt > score_neutre * 0.5:
        category = 'alternatif'
    elif total_agri > 0:
        category = 'neutre'
    else:
        category = 'autre'
    
    return {
        'category': category,
        'scores': scores,
        'matched_keywords': matched
    }


def classify_row(keywords_str: str) -> str:
    """Fonction simplifiée pour appliquer sur une colonne DataFrame."""
    keywords = parse_keywords(keywords_str)
    result = classify_discourse(keywords)
    return result['category']


def classify_row_detailed(keywords_str: str) -> dict:
    """Fonction détaillée retournant toutes les infos."""
    keywords = parse_keywords(keywords_str)
    return classify_discourse(keywords)


print("✅ Fonctions de classification définies !")
print(f"   - {len(KEYWORDS_PRODUCTIVISTE)} mots-clés productivistes")
print(f"   - {len(KEYWORDS_ALTERNATIF)} mots-clés alternatifs")
print(f"   - {len(KEYWORDS_AGRICULTURE_NEUTRE)} mots-clés agricoles neutres")

✅ Fonctions de classification définies !
   - 58 mots-clés productivistes
   - 70 mots-clés alternatifs
   - 83 mots-clés agricoles neutres


In [12]:
# Test sur 25 lignes
sample_df = df.head(25).copy()

# Appliquer la classification
sample_df['category_predicted'] = sample_df['nouns_in_keywords'].apply(classify_row)

# Afficher les résultats avec plus de détails
results = []
for idx, row in sample_df.iterrows():
    keywords = parse_keywords(row['nouns_in_keywords'])
    detailed = classify_discourse(keywords)
    results.append({
        'index': idx,
        'category_predicted': detailed['category'],
        'keywords_sample': keywords[:5] if keywords else [],  # Premiers 5 mots-clés
        'matched_prod': detailed['matched_keywords']['productiviste'][:3],
        'matched_alt': detailed['matched_keywords']['alternatif'][:3],
        'matched_neutre': detailed['matched_keywords']['neutre'][:3],
        'report_text_preview': row['report_text'][:100] + '...' if len(str(row['report_text'])) > 100 else row['report_text']
    })

results_df = pd.DataFrame(results)
print("📊 Résultats de classification sur 25 lignes:\n")
print(results_df[['index', 'category_predicted', 'keywords_sample']].to_string())
print("\n" + "="*60)
print("\n📈 Distribution des catégories:")
print(sample_df['category_predicted'].value_counts())

📊 Résultats de classification sur 25 lignes:

    index category_predicted                                           keywords_sample
0       0             neutre  [concurrence, stations, résultats, direction, colossaux]
1       1      productiviste             [temps, directeurs, retraites, site, viandes]
2       2             neutre        [heure, mérite, sympathie, euros, réglementations]
3       3             neutre                  [année, pommes, andalou, choses, limite]
4       4             neutre              [météo, choses, extérieur, règles, ensemble]
5       5      productiviste                      [choses, plantes, part, espace, sud]
6       6             neutre            [chaise, évident, année, frontalement, action]
7       7             neutre           [traditions, cochon, production, météo, achats]
8       8             neutre              [tests, effets, préservatif, peine, défense]
9       9             neutre                       [heure, pays, année, donald, quar

In [13]:
# Afficher les détails des mots-clés matchés pour mieux comprendre les classifications
print("🔍 Détail des mots-clés matchés par catégorie:\n")

for idx, row in sample_df.head(10).iterrows():
    keywords = parse_keywords(row['nouns_in_keywords'])
    detailed = classify_discourse(keywords)
    
    print(f"--- Ligne {idx} | Catégorie: {detailed['category'].upper()} ---")
    print(f"   Mots-clés productivistes: {detailed['matched_keywords']['productiviste']}")
    print(f"   Mots-clés alternatifs: {detailed['matched_keywords']['alternatif']}")
    print(f"   Mots-clés neutres (agri): {detailed['matched_keywords']['neutre']}")
    print(f"   Texte: {row['report_text'][:150]}...")
    print()

🔍 Détail des mots-clés matchés par catégorie:

--- Ligne 0 | Catégorie: NEUTRE ---
   Mots-clés productivistes: ['concurrence', 'production']
   Mots-clés alternatifs: ['abattage', 'rentables']
   Mots-clés neutres (agri): ['société', 'vache', 'machine', 'saison', 'précaution', 'marché', 'vaches', 'exploitation', 'producteur']
   Texte: plus vendu au monde a la une également cette ce troupeau de ce troupeau de vaches abattues dans le doubs pour stopper la propagation de la terre dans ...

--- Ligne 1 | Catégorie: PRODUCTIVISTE ---
   Mots-clés productivistes: ['cours', 'production']
   Mots-clés alternatifs: []
   Mots-clés neutres (agri): ['ventes', 'présentation', 'achat', 'prix']
   Texte: christo popov thomas fersen ces enseignants et cela on vous les angles et à déguster et on va voir si on s' est pas tremper son accuse bacha cela fait...

--- Ligne 2 | Catégorie: NEUTRE ---
   Mots-clés productivistes: []
   Mots-clés alternatifs: []
   Mots-clés neutres (agri): ['saison', 'prix'

In [14]:
# Appliquer la classification à tout le dataframe
print("🔄 Classification en cours sur tout le dataframe...")

df['category_predicted'] = df['nouns_in_keywords'].apply(lambda x: classify_row(x))

print(f"✅ Classification terminée sur {len(df)} lignes!")
print("\n📈 Distribution des catégories:")
print(df['category_predicted'].value_counts())
print("\n📊 Pourcentages:")
print(df['category_predicted'].value_counts(normalize=True).mul(100).round(1))

🔄 Classification en cours sur tout le dataframe...
✅ Classification terminée sur 2565 lignes!

📈 Distribution des catégories:
category_predicted
neutre           932
alternatif       874
productiviste    722
autre             37
Name: count, dtype: int64

📊 Pourcentages:
category_predicted
neutre           36.3
alternatif       34.1
productiviste    28.1
autre             1.4
Name: proportion, dtype: float64


In [15]:
# Sauvegarder le résultat
output_path = 'data/df_agri_keywords_classified.csv'
df.to_csv(output_path, index=False)
print(f"💾 Fichier sauvegardé: {output_path}")
print(f"   → {len(df)} lignes, {len(df.columns)} colonnes")
print(f"\n📋 Colonnes du fichier:")
print(df.columns.tolist())

💾 Fichier sauvegardé: data/df_agri_keywords_classified.csv
   → 2565 lignes, 18 colonnes

📋 Colonnes du fichier:
['channel_title', 'channel_name', 'country', 'segment_id', 'segment_start', 'segment_end', 'duration_seconds', 'num_keywords', 'avg_density', 'themes', 'report_text', 'category', 'secondary_categories', 'text_type', 'keywords', 'keywords_filtered', 'nouns_in_keywords', 'category_predicted']


In [16]:
df.head()

,channel_title,channel_name,country,segment_id,segment_start,segment_end,duration_seconds,num_keywords,avg_density,themes,report_text,category,secondary_categories,text_type,keywords,keywords_filtered,nouns_in_keywords,category_predicted
0,France 2,france2,france,5788,2025-12-03 06:34:04.031 +0100,2025-12-03 06:36:02.025 +0100,118,18,13.4,", attenuation_climatique_solutions_indirectes,...",plus vendu au monde a la une également cette c...,agriculture_alimentation,['energy'],segment,"['voyez', 'radiologues', 'entraîne activement ...","['entraîne activement lucie robert', 'stations...","['concurrence', 'stations', 'résultats', 'dire...",neutre
1,M6,m6,france,3445,2025-11-17 15:20:24.048 +0100,2025-11-17 15:21:26.064 +0100,62,4,5.3,", biodiversite_causes_indirectes, changement_c...",christo popov thomas fersen ces enseignants et...,agriculture_alimentation,[],segment,"['enseigne travaille', 'va voir si', 'david', ...","['enseigne travaille', 'cours', 'prochaine car...","['temps', 'directeurs', 'retraites', 'site', '...",productiviste
2,M6,m6,france,1379,2025-05-13 20:14:11.059 +0200,2025-05-13 20:15:50.004 +0200,99,8,8.1,"attenuation_climatique_solutions_indirectes, b...",la baignade tourne court sera <unk> occasion d...,agriculture_alimentation,[],segment,"['réglementations strictes imposées', 'essaie'...","['réglementations strictes imposées', 'parc na...","['heure', 'mérite', 'sympathie', 'euros', 'rég...",neutre
3,TF1,tf1,france,4852,2025-12-25 09:11:59.056 +0100,2025-12-25 09:13:11.082 +0100,72,15,17.2,"adaptation_climatique_solutions, attenuation_c...",le jardin ce ne sont pas uniquement des matièr...,agriculture_alimentation,[],segment,"['allez pouvoir', 'froid', 'herbes', 'myrtille...","['certains papiers', 'plantes', 'déchets', 'hu...","['année', 'pommes', 'andalou', 'choses', 'limi...",neutre
4,France 2,france2,france,5744,2025-11-30 07:36:06.090 +0100,2025-11-30 07:38:37.058 +0100,151,11,10.5,"biodiversite_causes, biodiversite_concepts_gen...",aux etats-unis que pensez-vous d' aller faire ...,agriculture_alimentation,[],segment,"['voyez', 'côté alpes', 'météo', 'peu gêné', '...","['côté alpes', 'météo', 'extérieur', 'choses',...","['météo', 'choses', 'extérieur', 'règles', 'en...",neutre
